In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
X = np.random.rand(10,2)

In [7]:
def L2_dist(a, b):
    return np.linalg.norm(a-b)

In [37]:
def compute_G0(X, d=L2_dist):
    W = np.zeros((len(X), len(X)))
    for i in range(len(X)-1):
        for j in range(i+1, len(X)):
            W[i, j] = d(X[i], X[j])
            W[j, i] = d(X[i], X[j])
    D_sqinv = np.diag(np.sum(W, axis=0) ** -0.5)
    return W, D_sqinv

In [38]:
W, D_sqinv = compute_G0(X)

In [45]:
def compute_laplacian(W, D_sqinv):
    return np.eye(len(W)) - D_sqinv @ W @ D_sqinv

In [47]:
L = compute_laplacian(W, D_sqinv)

In [ ]:
def compute_AM_normalised(L, D):
    return A